# Responsável por Importar Contratos do Cliente

## Importando bibliotecas

In [60]:
import sys, os

# Carregando pacote que estao fora de /transform
path_raiz = os.path.abspath(os.path.join("..", ".."))
sys.path.append(path_raiz)

In [61]:
import pandas as pd
from src.utils import carregar_tabela_excel
from src.insert.insert_contract import inserir_contrato
import json
import datetime

## Importando e tratando os dados

In [62]:
# Obter a data e hora atual
now = datetime.datetime.now()

# Formatar a data e hora como 'YYYYMMDD_HHMMSS'
timestamp = now.strftime("%Y%m%d_%H%M%S")
timestamp

'20250319_230411'

In [63]:
source_input = "../../data/baseTesteIXC_Transform.xlsx"
source_input_client_insertd = "../../src/inserted/cliente_insert_sucess.xlsx"
source_output = f"../../src/inserted/contrato_insert_sucess.xlsx"

In [64]:
# Importa tabela de cliente para puxar o id do cliente
df_client = pd.read_excel(source_input_client_insertd, dtype=str).fillna("")
df_client

,Unnamed: 0,id,razaoSocial,Email,Celular,cpf_cnpj,rg_ie,filial_id,idCidade,Endereco,Complemento,Numero,Bairro,Cep,idClienteIXC,statusInsercao,logRetorno
0,0,1306,Abelanio Veríssimo Peixoto,teste@gmail.com,99999999999,014.846.390-81,Isento,35,1659,Vila São Sebastião,Próximo ao cacimbão,SN,Centro,57830-000,117696,sucesso,"{'type': 'success', 'message': 'Registro inser..."


In [65]:
# Carregar o DataFrame com as colunas especificadas como strings
df = pd.read_excel(source_input, dtype=str).fillna("")

# Puxa o id do cliente para inserir no contrato tendo como chave o CPF ou CNPJ
df = df.merge(df_client[["idClienteIXC", "cpf_cnpj"]], how="left", left_on="cpf_cnpj", right_on="cpf_cnpj")
df

,Unnamed: 0,id,razaoSocial,Email,Celular,cpf_cnpj,rg_ie,filial_id,idCidade,Endereco,Complemento,Numero,Bairro,Cep,idPlanoIXC,idVencimentoIXC,Login,Senha,idClienteIXC
0,0,1306,Abelanio Veríssimo Peixoto,teste@gmail.com,99999999999,014.846.390-81,Isento,35,1659,Vila São Sebastião,Próximo ao cacimbão,SN,Centro,57830-000,6824,25,teste1@radios.com,peixoto,117696


In [66]:
# Lista das colunas que fazem sentido enviar para a API
colunas_para_api = [
    'idClienteIXC',
    'filial_id',
    'idCidade',
    'Endereco',
    'Numero',
    'Bairro',
    'Cep',
    'idPlanoIXC',
    'idVencimentoIXC',
    'Login',
    'Senha'
]

# Filtra o DataFrame para manter apenas essas colunas
df = df[colunas_para_api]


### Inserção teste

In [67]:
# inserir_contrato(
#     tipo="I",
#     id_cliente="117693",
#     id_vd_contrato="6824",  # Plano do contrato
#     id_tipo_contrato="5",  # Id tipo de cobrança
#     id_modelo="1",  # Id modelo do contrato
#     id_filial="35",
#     id_vendedor="1",
#     data_base_financeira="2024-04-01",
#     contrato="IMPORTAÇÃO",
#     id_carteira_cobranca="67",
#     cep="57160-000",
#     endereco="RUA DO TESTE",
#     numero="SN",
#     bairro="Bairro XPTO",
#     cidade="1659",
# )

## Inserção do DF

In [68]:
# Log para verificação de campo vazio
print(df.isnull().sum())
df

idClienteIXC       0
filial_id          0
idCidade           0
Endereco           0
Numero             0
Bairro             0
Cep                0
idPlanoIXC         0
idVencimentoIXC    0
Login              0
Senha              0
dtype: int64


,idClienteIXC,filial_id,idCidade,Endereco,Numero,Bairro,Cep,idPlanoIXC,idVencimentoIXC,Login,Senha
0,117696,35,1659,Vila São Sebastião,SN,Centro,57830-000,6824,25,teste1@radios.com,peixoto


### Adiciona o Nome Plano ao dataframe

In [69]:
import os
import sys

# Carregando pacote que estao fora de /transform
path_raiz = os.path.abspath(os.path.join("..", ".."))
sys.path.append(path_raiz)

from config.database_select import executa_query

#### Cria função somente para isso

In [70]:
# Função para adicionar a coluna 'nomePlano' ao DataFrame
def add_nomePlano_to_df(df):
    df = df.copy()  # Cria uma cópia do DataFrame para evitar problemas com slicing
    nomePlano_list = []
    for _, row in df.iterrows():
        idPlanoIXC = row['idPlanoIXC']
        # Monta a query utilizando o idPlanoIXC da linha
        query = f"""	
        SELECT
            vc.nome
        FROM
            vd_contratos vc 
        WHERE vc.id = {idPlanoIXC}
        """
        resultado = executa_query(query)
        # Caso a query retorne resultados, usamos o primeiro valor
        if resultado and len(resultado) > 0:
            id_plano = resultado[0][0]
        else:
            id_plano = None
        nomePlano_list.append(id_plano)
    
    # Cria a nova coluna 'nomePlano' no DataFrame utilizando .loc
    df.loc[:, 'nomePlano'] = nomePlano_list
    return df


In [71]:
df = add_nomePlano_to_df(df)
df

,idClienteIXC,filial_id,idCidade,Endereco,Numero,Bairro,Cep,idPlanoIXC,idVencimentoIXC,Login,Senha,nomePlano
0,117696,35,1659,Vila São Sebastião,SN,Centro,57830-000,6824,25,teste1@radios.com,peixoto,100MB_SPEEDY


## Insere Contrato

In [72]:
# Crie uma cópia do DataFrame original para evitar o SettingWithCopyWarning
df = df.copy()

# Crie as colunas adicionais (inicialmente vazias)
df.loc[:, "idContratoIXC"] = None
df.loc[:, "statusInsercao"] = None
df.loc[:, "logRetorno"] = None

for idx, row in df.iterrows():
    # Mapeamento dos dados do DataFrame para os parâmetros da função inserir_contrato.
    # OBSERVAÇÃO: Note que em id_cliente devemos utilizar o valor da coluna "idClienteIXC".
    # Se esse campo não existir ainda no df, certifique-se de que ele foi criado ou que o df já está enriquecido com o ID do cliente.
    id_cliente = row["idClienteIXC"]  # valor vindo da inserção anterior de cliente
    id_vd_contrato = row["idPlanoIXC"]  # mapeia para o plano do contrato
    id_tipo_contrato = row["idVencimentoIXC"]  # mapeia para o tipo de cobrança (conforme seu DF)
    id_filial = row["filial_id"]
    cidade = row["idCidade"]
    endereco = row["Endereco"]
    numero = row["Numero"]
    bairro = row["Bairro"]
    cep = row["Cep"]
    contrato = row["nomePlano"]  # mapeia para o nome do plano

    # Parâmetros fixos conforme o exemplo de chamada:
    tipo = "I"
    id_modelo = "1"               # Id modelo do contrato (fixo, conforme o exemplo)
    id_vendedor = "1"             # Id vendedor (fixo, conforme o exemplo)
    data_base_financeira = "01/01/2024"  # Data base financeira (pode ser fixo ou calculado conforme sua regra)
    descricao_aux_plano_venda = "IMPORTAÇÃO"       # Tipo de contrato (fixo)
    id_carteira_cobranca = "67"   # Id da carteira de cobrança (fixo)

    # Chamamos a função inserir_contrato com os parâmetros mapeados:
    response = inserir_contrato(
        tipo=tipo,
        id_cliente=id_cliente,
        id_vd_contrato=id_vd_contrato,
        id_tipo_contrato=id_tipo_contrato,
        id_modelo=id_modelo,
        id_filial=id_filial,
        id_vendedor=id_vendedor,
        data_base_financeira=data_base_financeira,
        contrato=contrato,
        descricao_aux_plano_venda=descricao_aux_plano_venda,
        id_carteira_cobranca=id_carteira_cobranca,
        cep=cep,
        endereco=endereco,
        numero=numero,
        bairro=bairro,
        cidade=cidade,
    )

    status_code = response.status_code
    content = response.content  # bytes

    # Fazemos parse do JSON retornado:
    json_resp = json.loads(content.decode("utf-8", errors="replace"))

    if status_code == 200 and json_resp.get("type") == "success":
        # Sucesso: capturamos o "id" do contrato inserido
        contrato_id = json_resp.get("id")  # Exemplo: "154036"
        df.at[idx, "idContratoIXC"] = contrato_id
        df.at[idx, "statusInsercao"] = "sucesso"
        # Em caso de sucesso, o log pode armazenar o JSON completo ou ser deixado vazio
        df.at[idx, "logRetorno"] = json_resp
    else:
        # Em caso de falha: registra os logs e garante que idContratoIXC fique como None
        df.at[idx, "idContratoIXC"] = None
        df.at[idx, "statusInsercao"] = "falha"
        df.at[idx, "logRetorno"] = json_resp


200 OK
b'{"type":"success","message":"Registro inserido com sucesso!","id":"154047","atualiza_campos":[{"tipo":"i","campo":"data_desistencia","valor":""},{"tipo":"s","campo":"status","valor":"P"},{"tipo":"i","campo":"id_cliente","valor":"117696"},{"tipo":"i","campo":"data_ativacao","valor":""},{"tipo":"d","campo":"data_renovacao","valor":""},{"tipo":"d","campo":"nao_avisar_ate","valor":""},{"tipo":"d","campo":"nao_bloquear_ate","valor":""},{"tipo":"d","campo":"data_cancelamento","valor":""},{"tipo":"d","campo":"dt_ult_bloq_manual","valor":""},{"tipo":"d","campo":"data_cadastro_sistema","valor":"2025-03-19"}]}'


### Retorno

In [73]:
df.to_excel(source_output)

In [74]:
df

,idClienteIXC,filial_id,idCidade,Endereco,Numero,Bairro,Cep,idPlanoIXC,idVencimentoIXC,Login,Senha,nomePlano,idContratoIXC,statusInsercao,logRetorno
0,117696,35,1659,Vila São Sebastião,SN,Centro,57830-000,6824,25,teste1@radios.com,peixoto,100MB_SPEEDY,154047,sucesso,"{'type': 'success', 'message': 'Registro inser..."
